In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import datetime as dt
from datetime import datetime
from pandas import to_datetime as to_datetime
import os
import re

#notebook to calculate CF for all the stations based on a specific timestep

~ suppose we are going to get CF based on 2020/08/01 14:00 
~ the input file for nwp should be the most recent run time so in this case 202008010000_nwp_t2.csv
~ we must also account for not taking any values in the first six hours after the runtime in any calculation this is why we cant chose to open 202008011200_nwp_t2.csv even though its the most recent run.
~ the input file would always be obs_t2.csv (refer to readme.md)


In [ ]:
#open obs csv and set a datetime index (UTC)
obs_input = pd.read_csv('path to csv')
start_date = pd.to_datetime('2020-07-28 16:00') #convert 2020/07/30 00:00 to UTC = 2020/07/28 16:00
end_date = start_date + pd.DateOffset(hours=767)
date_index = pd.date_range(start=start_date, end=end_date, freq='H')

#open nwp csv
nwp_input = pd.read_csv('path to csv')

#open staioon_dictionary
station_dictionary = pd.read_csv('./input/latlonstation.csv')

In [ ]:
#DEFINE THE TIME TO GET CF FOR
CF_spec_time = '2020-08-01 14:00'
CF_spec_time = datetime.strptime(CF_spec_time, '%Y-%m-%d %H:%M')
CF_spec_time


In [ ]:

#define time for obs
start_obs_time = CF_spec_time - pd.DateOffset(hours = 6) #to get mean of the previous six hours
end_obs_time = CF_spec_time
#define times for nwp #no need to get mean of 6 hours for nwp
nwp_time =  CF_spec_time


In [ ]:
#assign empty arrays to fill in for calcualuted CF for each station
CF_array = np.array([])

now we start the loop for calcualting CF

In [ ]:
station_id = station_dictionary['station_id']
for x, id in enumerate(station_id):
        # extract temperatures from each station ID for OBS
        y = obs_input['%s' % id]
        # print(y)
        obs = y[start_obs_time:end_obs_time]
        obs_original = obs.iloc[13]
        obs = obs.iloc[6:11].values  # T2 of (31/0200:31/0700 BJT) of OBS
        obs = obs.mean()
        # extract temperatures from each station ID for NWP
        z = nwp_input['%s' % id]
        # print(z)
        nwp = z[nwp_time]
        #compute CF and fill empty array 
        CF = obs-nwp
        CF_array = np.append(CF_array, CF)

#create a dataframw with only station_id, lat and lon
CF_df = station_dictionary.drop(station_dictionary.loc[:, 'x':'y':'alt'].columns,axis = 1)
CF_df.insert(4, "CF", CF_array)
print(CF_df)